# 05_A_Software_Comparison_PointCloud

In diesem Jupyter Notebook werden die Punkwolken der Softwareprodukte "Agisoft Metashape" und "Drone2Map" miteinander verglichen. 
Die Daten basieren auf Bilder der DJI Mini 3 Pro von den folgenden Aufnahmetagen:
11.03.2025 

### Import der Libraries

In [2]:
import pandas as pd
import os
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import laspy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from matplotlib.ticker import FuncFormatter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Daten einlesen
df = pd.read_csv("RMSE_Analyse.csv")

# Stil setzen
sns.set(style="whitegrid")

# Balkendiagramm: RMSE_Total_mm nach Software
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x="Aufnahme_ID", y="RMSE_Total_mm", hue="Software")
plt.title("RMSE Total Vergleich (mm)")
plt.ylabel("RMSE Total [mm]")
plt.xlabel("Aufnahme")
plt.legend(title="Software")
plt.tight_layout()
plt.show()

# Balkendiagramm: RMSE_X, Y, Z
rmse_xyz = df.melt(id_vars=["Aufnahme_ID", "Software"], 
                   value_vars=["RMSE_X_mm", "RMSE_Y_mm", "RMSE_Z_mm"],
                   var_name="Achse", value_name="RMSE [mm]")

plt.figure(figsize=(10, 6))
sns.barplot(data=rmse_xyz, x="Aufnahme_ID", y="RMSE [mm]", hue="Software", palette="Set2", ci=None, dodge=True)
plt.title("RMSE pro Achse (X, Y, Z) nach Software")
plt.xlabel("Aufnahme")
plt.xticks(rotation=15)
plt.legend(title="Software")
plt.tight_layout()
plt.show()

# RMSE Pixelvergleich
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x="Aufnahme_ID", y="RMSE_Pixel", hue="Software", palette="pastel")
plt.title("RMSE Projektion (Pixel)")
plt.ylabel("RMSE [Pixel]")
plt.xlabel("Aufnahme")
plt.tight_layout()
plt.show()